In [ ]:
# alle notwendigen Imports 
import torch
from IPython.display import Image  # für Bilder anzeigen 
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import cv2

random.seed(108)

In [ ]:

# Sammelt Daten aus dem PASCAL VOC Format und gibt diese als python dict zurück 
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Erstellung von python dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # XML File wird geladen und relevante Größen herausgeschrieben 
    for elem in root:
        # get Filename
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # get image_size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # get bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    # gibt das python dict mit allen wichtigen Infos zurück 
    return info_dict

In [ ]:
# Testen der Funktion 
path_to_xml_file = "C:/Users/tabby/Desktop/Informatik/Double-KI/Yolo-Dobble Dataset/raw_data/card0.xml"
print(extract_info_from_xml(path_to_xml_file))

In [ ]:
# python dict mit allen Symbolen (stammt aus anderem Code, deswegen mit Pfaden)
icons = {
    "heart_icon": 'images/big pictures/350_heart.png',
    "sun_glass_icon": ('images/big pictures/350_sunglass.png'),
    "raindrop_icon": ('images/big pictures/350_raindrop.png'),
    "hand_icon": ('images/big pictures/350_hand.png'),
    "hammer_icon": ('images/big pictures/350_hammer.png'),
    "bird_icon": ('images/big pictures/350_bird.png'),
    "stop_icon": ('images/big pictures/350_stopp.png'),
    "clock_icon": ('images/big pictures/350_clock.png'),
    "net_icon": ('images/big pictures/350_net.png'),
    "moon_icon": ('images/big pictures/350_moon.png'),
    "men_icon": ('images/big pictures/350_men.png'),
    "eye_icon": ('images/big pictures/350_eye.png'),
    "yingyang_icon": ('images/big pictures/350_yingyang.png'),
    "fire_icon": ('images/big pictures/350_fire.png'),
    "exclamation_icon": ('images/big pictures/350_exclamation.png'),
    "anker_icon": ('images/big pictures/350_anker.png'),
    "apple_icon": ('images/big pictures/350_apple.png'),
    "bomb_icon": ('images/big pictures/350_bomb.png'),
    "bulb_icon": ('images/big pictures/350_bulb.png'),
    "cactus_icon": ('images/big pictures/350_cactus.png'),
    "candle_icon": ('images/big pictures/350_candle.png'),
    "car_icon": ('images/big pictures/350_car.png'),
    "carrot_icon": ('images/big pictures/350_carrot.png'),
    "cat_icon": ('images/big pictures/350_cat.png'),
    "cheese_icon": ('images/big pictures/350_cheese.png'),
    "clef_icon": ('images/big pictures/350_clef.png'),
    "cloverleaf_icon": ('images/big pictures/350_cloverleaf.png'),
    "cube_icon": ('images/big pictures/350_cube.png'),
    "dino_icon": ('images/big pictures/350_dino.png'),
    "dog_icon": ('images/big pictures/350_dog.png'),
    "dolphin_icon": ('images/big pictures/350_dolphin.png'),
    "dragon_icon": ('images/big pictures/350_dragon.png'),
    "horse_icon": ('images/big pictures/350_horse.png'),
    "igloo_icon": ('images/big pictures/350_igloo.png'),
    "key_icon": ('images/big pictures/350_key.png'),
    "ladybug_icon": ('images/big pictures/350_ladybug.png'),
    "lightning_icon": ('images/big pictures/350_lightning.png'),
    "lips_icon": ('images/big pictures/350_lips.png'),
    "lock_icon": ('images/big pictures/350_lock.png'),
    "maple_icon": ('images/big pictures/350_maple.png'),
    "pen_icon": ('images/big pictures/350_pen.png'),
    "question_icon": ('images/big pictures/350_question.png'),
    "scissors_icon": ('images/big pictures/350_scissors.png'),
    "skull_icon": ('images/big pictures/350_skull.png'),
    "snowman_icon": ('images/big pictures/350_snowman.png'),
    "spider_icon": ('images/big pictures/350_spider.png'),
    "splotch_icon": ('images/big pictures/350_splotch.png'),
    "sun_icon": ('images/big pictures/350_sun.png'),
    "target_icon": ('images/big pictures/350_target.png'),
    "tree_icon": ('images/big pictures/350_tree.png'),
    "zebra_icon": ('images/big pictures/350_zebra.png'),
    "clown_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_clown.png'),
    "snow_flake_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_snow_flake.png'),
    "turtle_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_turtle.png'),
    "flower_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_flower.png'),
    "bottle_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_bottle.png'),
    "ghost_icon": ('C:/Users/tabby/Desktop/Informatik/Double-KI/images/350_ghost.png'),
}

# Pfade werden entfernt und mit einem Index gekennzeichnet 
i = 0
for icon in icons:
    icons[icon] = i
    i += 1

print("Icons", icons)
print("len", len(icons))

In [ ]:
# aus vorherigem Code 
class_name_to_id = icons 

# Funktion extract_info_from_xml() wird genutzt
# Wichtige Daten werden 
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # Jede einzelne Detektion in einer Karte 
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id[b["class"]]
        except KeyError:
        # falls Fehler auftreten 
            print(f"This class "{b["class"]}" doesn't exist")
        
        # Yolov5 benötigt die Koordinaten aus einem anderen Blickpunkt (mittlere Werte)
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Koordinaten werden nicht als Pixel angegeben, sondern relativ zur Bildgröße als Dezimalwert  
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        # Daten werden in das richtige Format gebracht 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Neuer Dateiname mit Pfad wird aus altem Pfad erstellt
    save_file_name = os.path.join("images/annotations", info_dict["filename"].replace("png", "txt"))
#     print("save_file_name", save_file_name)
#     print("print buffer", print_buffer)
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file = open(save_file_name, "w"))

In [ ]:
# Get the annotations
path_to_pascal_voc_format_files = 'C:/Users/tabby/Desktop/Informatik/Double-KI/yolov5/dobble_ai/raw_data/'
annotations = [(path_to_pascal_voc_format_files + x) for x in os.listdir(path_to_pascal_voc_format_files) if x[-3:] == "xml"]
annotations.sort()
# print(annotations)

# Konvertierung und speichern der annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
new_path_for_yolov5_annotations = "C:/Users/tabby/Desktop/Informatik/Double-KI/yolov5/dobble_ai/images/annotations/"
annotations = [(+ x) for x in os.listdir(new_path_for_yolov5_annotations) if x[-3:] == "txt"]

In [ ]:
random.seed(random.randint(1, 100))

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        color = tuple((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
#         color =  tuple(np.random.random(size=3) * 256)
#         color = tuple(np.random.randint(size=3) * 256)
        
#         print(color)
        plotted_image.rectangle(((x0,y0), (x1,y1)), width=5, outline=color)

        plotted_image.text( (x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))], stroke_width=3, fill=color)
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

# Get the corresponding image file
image_file = annotation_file.replace("annotations", "test").replace("txt", "png")
print("image_file", image_file)
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)